# Corpus Analysis
Elementary analysis on the final and cleaned splits on the laws from South Carolina.

In [1]:
import os
import pandas as pd
import numpy as np

from sys import stderr
from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import tqdm  # For printing out progress bar
# from tqdm.auto import tqdm  # For printing out progress bar

from nltk.corpus import stopwords # Import the stop word list
pd.set_option('display.max_colwidth', None)

In [77]:
stop_words = stopwords.words('english')

# Add some custom words to the list
stop_words.append('said')
stop_words.append('shall')
stop_words.append('ee')
stop_words.append('00')
stop_words.append('state')
stop_words.append('may')
stop_words.append('src')
stop_words.append('sec')
stop_words.append('sec.')
stop_words.append('town')
stop_words.append('section')
stop_words.append('county')
stop_words.append('act')
stop_words.append('board')
stop_words.append('000')
stop_words.append(';')
stop_words.append('approved')
stop_words.append('one')
stop_words.append('general')
stop_words.append('upon')
stop_words.append('hereby')

In [78]:
# Read all folder names in the OCR (or a specified) directory
ocred_path = '/work/otb-lab/OCRed'
# ocred_path = '/Users/nitingupta/Desktop/OTB/OCRed'

years = [name for name in os.listdir(ocred_path) if not name.startswith('.')]
years.sort()
print(years)

['1868-69', '1869-1870', '1870-1871', '1871', '1871-1872', '1872-1873', '1873', '1873-1874', '1874', '1875-76', '1877-78', '1878', '1879', '1880', '1881-82', '1883', '1884', '1885', '1886-1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1958b', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968']


Read the final split csv...

In [6]:
# df = pd.read_csv('../Splitting/final-update/final-splits/final_splits.csv', index_col = 0)
df = pd.read_csv('../split/updated/results/final_splits.csv', index_col = 0)

In [7]:
df['year'] = df.index.str.split("_").str[0]

In [8]:
df

,law_type,state,sentence,length,start_page,end_page,act,section,path,year
id,,,,,,,,,,
1868-69_0000,Act,SOUTH CAROLINA,"AN ACT ACCEPTING THE BENEFITS OF “AN ACT DONATING PUBLIC LANDS TO THE SEVERAL STATES AND TERRITORIES WHICH MAY PROVIDE COLLEGES FOR THE BENEFIT OF AGRICULTURE AND THE MECHANIC ARTS,” APPROVED THE SECOND DAY OF JULY, IN THE YEAR OF OUR LORD ONE THOUSAND EIGHT HUNDRED AND SIXTYTWO.",285,71,71,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0001,Act,SOUTH CAROLINA,"Whereas, by an Act of Congress, approved the twentythird day of July, in the year of our Lord one thousand eight hundred and sixtysix, entitled “ An Act to amend the fifth Section of an Act entitled ‘An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,’” approved the second day of July, ig the year of our Lord one thousand eight hundred and sixtytwo, so as to extend the time within which the provisions of said Act shall be accepted and such colleges established, it was, among other things, by the Senate and House of Representatives of the United States of America, in Congress assembled, enacted that the time in which the several States may comply with the provisions of the said Act of July second, eighteen hundred and sixtytwo, entitled “An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,” is extended so that the acceptance of the benefits of the said Act may be expressed within three years from the passage of the Act first above mentioned: Section 1.",1161,71,71,1,0,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0002,Act,SOUTH CAROLINA,"Beit enacted by the Senate and House of Representatives of the State of South Carolina, now met and sitting in General Assembly, and by the authority of the same, That the State of South Carolina does hereby express its acceptance of the benefitsof the said Act of Congress, approved on the second day of July, in the year of our Lord one thousand eight hundred and sixtytwo, entitled “An Act donating public lands to the several States and Territories which may provide colleges for the benefit of agriculture and the mechanic arts,” and does hereby assent to the provisions in said Act contained, and to the conditions on which the grant of land and scrip by said Act authorized is made, and binds herself to the faithful performance of all the stipulations by her to be assumed in said Act contained; and it is further desired, that the State may be allowed to use the same for the establishment and support of a system of common free schools, if the State may so desire.",979,71,72,1,1,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/071.jpg,1868-69
1868-69_0003,Act,SOUTH CAROLINA,"Upon the passage of this Act, the Governor of the State is authorized to take such measures as he may deem necessary to secure the early realization of the benefits of the Act above mentioned.",197,72,72,1,2,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/072.jpg,1868-69
1868-69_0004,Act,SOUTH CAROLINA,"In the Senate House, the twentysecond day of July, in the year of our Lord one thousand eight hundred and sixtyeight.",122,72,72,1,1,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1868-69/images/072.jpg,1868-69
...,...,...,...,...,...,...,...,...,...,...
1968_6685,Act,SOUTH CAROLINA,Buy and sell real estate.,30,1000,1000,1374,8,https://emailsc.sharepoint.com/:i:/r/sites/COTEAM-ULIB-OntheBooks/Shared%20Documents/General/OCRed/1968/images/1000.tiff,1968
1968_6686,Act,SOUTH CAROLINA,"Section 142563.1 of the

<br>

## Get Most Frequent
Make a dataframe that contains most frequent words (along with their counts) for each year.

In [69]:
# Define the number of top terms to retrieve
top_n = 5

# Define the minimum length of characters for each word
min_len = 5

In [79]:
# Create an empty list to store dictionaries for each year's data
data_to_concat = []

# Group the DataFrame by 'year'
grouped = df.groupby('year')

# Set up the progress bar
progress_bar = tqdm(total=len(years), file=stderr)

# Iterate through each group (year)
for year, group in grouped:
    
    # Update the progress bar
    progress_bar.set_description(f"Processing year {year}")
        
    # Concatenate all sentences for the current year into one document
    year_text = ' '.join(group['sentence'])
    
    # Get the top n frequent words
    words_lower = pd.Series(year_text.lower().split())
    # Get words which are not in stop words and are greater than the min length
    top_frequent = dict(words_lower[
        ((words_lower.str.len() > min_len) & (~words_lower.isin(stop_words)))
    ].value_counts()[:top_n])
    
    # Create a dictionary for the current year's data
    year_data = {'year': year, 'top frequent': top_frequent}
    
    # Append the dictionary to the list
    data_to_concat.append(year_data)
        
    # Update the progress bar
    progress_bar.update(1)
    
# Close the progress bar
progress_bar.set_description(f"Processed the list")
progress_bar.close()
    
# Concatenate all dictionaries into a single DataFrame
df_analysis = pd.DataFrame(data_to_concat)
df_analysis.set_index('year', inplace=True)

Processed the list: 100%|██████████| 100/100 [00:09<00:00, 10.73it/s]      


In [80]:
df_analysis

,top frequent
year,
1868-69,"{'auditor': 207, 'property': 207, 'person': 197, 'state,': 177, 'county,': 139}"
1869-1870,"{'company': 170, 'authority': 157, 'carolina,': 151, 'thousand': 151, 'hundred': 142}"
1870-1871,"{'council': 302, 'intendant': 302, 'within': 290, 'public': 228, 'persons': 223}"
1871,"{'carolina,': 811, 'authority': 810, 'thousand': 750, 'hundred': 747, 'within': 737}"
1871-1872,"{'authority': 280, 'carolina,': 279, 'thousand': 270, 'hundred': 266, 'within': 257}"
...,...
1964,"{'amended': 531, 'person': 409, 'approval': 392, 'district': 391, 'effect': 374}"
1965,"{'laws1965': 1674, 'district': 1452, 'commission': 1428, 'provided,': 1239, 'school': 1001}"
1966,"{'amended': 660, 'commission': 631, 'office': 490, 'carolina,': 486, 'school': 464}"


<br>

## Get TF-IDF Scores for Bigrams and Trigrams

Make a dataframe that contains the top tf-idf words (along with their scores) for each year.
<br>Furthermore, consider bigrams and trigrams for the analysis.

In [82]:
# Create an empty list to store dictionaries for each year's data
ngrams = [2, 3]

for ngram in ngrams:
    
    data_to_concat = []

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words = stop_words, ngram_range = (ngram, ngram))

    # Fit the vectorizer on all sentences
    tfidf_vectorizer.fit(df['sentence'])

    # Transform the sentences into TF-IDF vectors
    tfidf_matrix = tfidf_vectorizer.transform(df['sentence'])

    # Group the DataFrame by 'year'
    grouped = df.groupby('year')

    # Set up the progress bar
    progress_bar = tqdm(total=len(years), file=stderr)

    # Iterate through each group (year)
    for year, group in grouped:

        # Update the progress bar
        progress_bar.set_description(f"Processing year {year} for ngram {ngram}")

        # Concatenate all sentences for the current year into one document
        year_text = ' '.join(group['sentence'])

        # Transform the concatenated text into a TF-IDF vector
        year_tfidf_vector = tfidf_vectorizer.transform([year_text])

        # Get the TF-IDF scores for the year
        year_tfidf_scores = year_tfidf_vector.toarray()[0]

        # Get the indices of the top N terms for the year
        ## row.argsort() calculates the indices that would sort the elements in ascending order
        ## [-top_n:] takes the last top_n elements from the sorted indices array
        ## [::-1] reverses the order of the selected indices to make the indices in descending order.
        top_indices = year_tfidf_scores.argsort()[-top_n:][::-1]

        # Get the top terms and their TF-IDF scores
        top_terms = [tfidf_vectorizer.get_feature_names_out()[i] for i in top_indices]
        top_scores = [year_tfidf_scores[i] for i in top_indices]

        # Round top_scores to 3 decimal places
        top_scores = list(np.around(np.array(top_scores),3))

        # Create a dictionary for the current year's data
        # year_data = {f'top {ngram}-gram tf-idf': dict(zip(top_terms, top_scores))}
        year_data = dict(zip(top_terms, top_scores))

        # Append the dictionary to the list
        data_to_concat.append(year_data)

        # Update the progress bar
        progress_bar.update(1)

    # Close the progress bar
    progress_bar.set_description(f"Processed the list for ngram {ngram}")
    progress_bar.close()

    # Concatenate all dictionaries into a single DataFrame
    df_analysis[f'top {ngram}-gram tf-idf'] = data_to_concat

Processed the list for ngram 3: 100%|██████████| 100/100 [36:52<00:00, 22.12s/it]      


In [83]:
df_analysis

,top frequent,top 2-gram tf-idf,top 3-gram tf-idf
year,,,
1868-69,"{'auditor': 207, 'property': 207, 'person': 197, 'state,': 177, 'county,': 139}","{'house representatives': 0.162, 'senate house': 0.161, 'real estate': 0.159, 'hundred sixtyeight': 0.12, 'south carolina': 0.111}","{'eight hundred sixtyeight': 0.138, 'senate house representatives': 0.136, 'enacted senate house': 0.129, 'house representatives south': 0.126, 'thousand eight hundred': 0.124}"
1869-1870,"{'company': 170, 'authority': 157, 'carolina,': 151, 'thousand': 151, 'hundred': 142}","{'house representatives': 0.205, 'carolina met': 0.2, 'senate house': 0.199, 'met sitting': 0.195, 'representatives south': 0.188}","{'south carolina met': 0.229, 'senate house representatives': 0.228, 'carolina met sitting': 0.224, 'house representatives south': 0.215, 'representatives south carolina': 0.214}"
1870-1871,"{'council': 302, 'intendant': 302, 'within': 290, 'public': 228, 'persons': 223}","{'intendant wardens': 0.28, 'march 1871': 0.169, 'house representatives': 0.147, 'senate house': 0.146, 'carolina met': 0.145}","{'senate house representatives': 0.18, 'carolina met sitting': 0.178, 'south carolina met': 0.178, 'met sitting assembly': 0.174, 'house representatives south': 0.172}"
1871,"{'carolina,': 811, 'authority': 810, 'thousand': 750, 'hundred': 747, 'within': 737}","{'carolina met': 0.203, 'intendant wardens': 0.201, 'enacted senate': 0.196, 'senate house': 0.191, 'south carolina': 0.19}","{'south carolina met': 0.266, 'enacted senate house': 0.238, 'carolina met sitting': 0.231, 'met sitting assembly': 0.177, 'senate house representatives': 0.174}"
1871-1872,"{'authority': 280, 'carolina,': 279, 'thousand': 270, 'hundred': 266, 'within': 257}","{'intendant wardens': 0.197, 'carolina met': 0.183, 'met sitting': 0.182, 'enacted senate': 0.182, 'assembly authority': 0.18}","{'south carolina met': 0.222, 'carolina met sitting': 0.222, 'enacted senate house': 0.219, 'senate house representatives': 0.208, 'met sitting assembly': 0.206}"
...,...,...,...
1964,"{'amended': 531, 'person': 409, 'approval': 392, 'district': 391, 'effect': 374}","{'1962 code': 0.406, 'code amended': 0.224, 'effect approval': 0.157, 'april 1964': 0.15, 'take effect': 0.148}","{'1962 code amended': 0.305, 'effect approval governor': 0.209, 'take effect approval': 0.208, 'day april 1964': 0.198, '1962 code relating': 0.173}"
1965,"{'laws1965': 1674, 'district': 1452, 'commission': 1428, 'provided,': 1239, 'school': 1001}","{'temporary laws1965': 0.281, 'permanent laws1965': 0.248, 'south carolina': 0.184, 'conservation district': 0.176, 'carolina 1962': 0.168}","{'south carolina 1962': 0.233, 'watershed conservation district': 0.188, 'laws south carolina': 0.171, 'loca temporary laws1965': 0.17, 'code laws south': 0.168}"
1966,"{'amended': 660, 'commission': 631, 'office': 490, 'carolina,': 486, 'school': 464}","{'carolina 1962': 0.289, 'south carolina': 0.239, 'laws south': 0.214, 'code laws': 0.208, '1962 code': 0.158}","{'south carolina 1962': 0.357, 'laws south carolina': 0.265, 'code laws south': 0.26, 'effect approval governor': 0.186, 'take effect approval': 0.184}"


In [84]:
df_analysis.to_csv('./analysis.csv')

<br>